In [1]:
%pip install -U classiq

     |████████████████████████████████| 279 kB 6.3 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 13.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 8.1 MB/s eta 0:00:01
     |████████████████████████████████| 15.3 MB 169.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 111.4 MB/s eta 0:00:01
     |████████████████████████████████| 99 kB 69.0 MB/s  eta 0:00:01
     |████████████████████████████████| 96 kB 48.5 MB/s  eta 0:00:01
     |████████████████████████████████| 70 kB 42.6 MB/s eta 0:00:011
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
     |████████████████████████████████| 80 kB 12.1 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 21.7 MB/s eta 0:00:011
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached PyJWT-2.6.0-py3-none-any.whl (20 kB)
     |████████████████████████████████| 166 kB 49.5 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 31.6 MB/s  eta 0:00:01
  

     |████████████████████████████████| 151 kB 28.4 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 10.9 MB/s eta 0:00:01
  Created wheel for Pyomo: filename=Pyomo-6.0.1-py2.py3-none-any.whl size=2470801 sha256=38dc43100f8febca5eeb3131b98ace8fec90907b6f0132d9b95086490c1984d9
  Stored in directory: /Users/youfuqian/Library/Caches/pip/wheels/1a/07/05/11c7e9af931cf1f17f9fa91d5d884a203d6cde6f4875cea144
Successfully built Pyomo
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.0
    Uninstalling numpy-1.23.0:
      Successfully uninstalled numpy-1.23.0
  Attempting uninstall: sympy
    Found existing installation: sympy 1.10.1
    Uninstalling sympy-1.10.1:
      Successfully uninstalled sympy-1.10.1
  Attempting uninstall: scipy
    Found existing installation: scipy 1.8.1
    Uninstalling scipy-1.8.1:
      Successfully uninstalled scipy-1.8.1
  Attempting uninstall: networkx
    Found existing installation: networkx 2.8.4
    Uninstalling networkx-2

In [1]:
import qiskit
from qiskit import assemble, QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit.compiler import transpile
from random import *
from qiskit import Aer
sim = Aer.get_backend("statevector_simulator")
import numpy as np
import pandas as pd
from qiskit.circuit.library.standard_gates import HGate
from qiskit.circuit.library import RZGate, RYGate
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
def neighbor_list(i, j, N):
  a = i-1
  b = j-1
  c = i+1 
  d = j+1 
  if a < 0: 
      a = N-1
  if b < 0: 
      b = N-1
  if c > N-1: 
      c = 0 
  if d > N-1: 
      d = 0
  left   = (a, j)
  above  = (i, b)
  right  = (c, j)
  below  = (i, d)
  return [left, above, right, below]

In [3]:
spin, N = 16, 4

In [4]:
def single_step_qc(i, temp): 
    global N, spin 
    prob1= np.exp(-8/temp)
    angle1 = 2*np.arccos(np.sqrt(prob1))
    prob2 = np.exp(-4/temp)
    angle2 = 2*np.arccos(np.sqrt(prob2))
    ### construct neighbor qubit 
    ising = np.arange(0, N**2).reshape(N,N)
    index = np.where(i==ising)
    neighbor_qubit = []
    for k in neighbor_list(index[0], index[1],N):
        neighbor_qubit.append(ising[k][0])
    ### construct circuit 
    qc = QuantumCircuit(spin+4)
    aux_qubit = [spin, spin+1, spin+2, spin+3]
    # flip first 
    qc.x(i)
    qc.barrier()
    for j in range(4): 
        qc.cx(i, aux_qubit[j])
        qc.cx(neighbor_qubit[j], aux_qubit[j])
    # rotate or not depends on sign
    #####################situation 1 : all same signs #####################
    [aux1, aux2, aux3, aux4] = aux_qubit
    # default is ferromagnetic
    mcry= RYGate(angle1).control(4,label=None )
    qc.append(mcry,[aux1, aux2, aux3, aux4, i])
    #####################situation 2 : 3 same signs, 1 different signs #####################
    mcry_1= RYGate(angle2).control(4,label=None )
    mcry_2= RYGate(angle2).control(4,label=None )
    mcry_3= RYGate(angle2).control(4,label=None )
    mcry_4= RYGate(angle2).control(4,label=None )
    mcry_1._ctrl_state=14
    mcry_2._ctrl_state=13
    mcry_3._ctrl_state=11
    mcry_4._ctrl_state=7
    qc.append(mcry_1, [aux1, aux2, aux3, aux4, i])
    qc.append(mcry_2, [aux1, aux2, aux3, aux4, i])
    qc.append(mcry_3, [aux1, aux2, aux3, aux4, i])
    qc.append(mcry_4, [aux1, aux2, aux3, aux4, i])
    qc.barrier()
    return qc

In [5]:
def cal_M(state):
    M = 0
    for i in state:
        if i == '1': 
            M += 1
        else: 
            M -= 1 
    return M/16

In [6]:
def phase_space_sweep(temp, circuit_size = 32):
    # MC_sweep
    qc = QuantumCircuit(spin+4)
    # for 4x4 ising model at least need to repeat 16 circuits to get all basis (2**16)
    # I used 32 to make sure that we can cover the whole phase space
    # could change this number
    for _ in range(circuit_size):
        pick = randint(0, spin-1)
        qc.compose(single_step_qc(pick, temp), inplace=True)
    return qc

In [12]:
def measure(circuit):
    circuit.remove_final_measurements()
    circuit.measure_all()
    backend = Aer.get_backend('aer_simulator')
    circuit = transpile(circuit, backend )
    final_counts = backend.run(circuit, shots=8192).result().get_counts()
    state = max(final_counts, key= final_counts.get)[4:]
    mag = abs(cal_M(state))
    #print("Most probably state: ", state)
    #print("Magnetization: ", mag) 
    return state, mag

In [ ]:
for _ in range(5):
    qc = phase_space_sweep(4)
    measure(qc)

In [8]:
# Average of many results from circuit size 32
temps = np.linspace(0.01, 15, 20)
df = pd.DataFrame()

for temp in temps:
    magnetization = []
    for i in range(10):
        qc = phase_space_sweep(temp)
        state, mag = measure(qc)
        magnetization.append(mag)
    df['kT='+str(temp)] = magnetization

df.to_csv('Circuit 32_Samples 10.csv')

In [9]:
# Average of many results from circuit size 32
temps = np.linspace(0.01, 15, 20)
df = pd.DataFrame()

for temp in temps:
    magnetization = []
    for i in range(10):
        qc = phase_space_sweep(temp, 64)
        state, mag = measure(qc)
        magnetization.append(mag)
    df['kT='+str(temp)] = magnetization

df.to_csv('Circuit 64_Samples 10.csv')

In [10]:
#Circuit size 256
temps = np.linspace(0.01, 15, 20)
df = pd.DataFrame()

magnetization = []
for temp in temps:
    qc = phase_space_sweep(temp, 256)
    state, mag = measure(qc)
    magnetization.append(mag)
df['kT'] = temps
df['Magnetization'] = magnetization

df.to_csv('Circuit 256_Samples 1.csv')

In [11]:
#Circuit size 512
temps = np.linspace(0.01, 15, 20)
df = pd.DataFrame()

magnetization = []
for temp in temps:
    qc = phase_space_sweep(temp, 512)
    state, mag = measure(qc)
    magnetization.append(mag)
df['kT'] = temps
df['Magnetization'] = magnetization

df.to_csv('Circuit 512_Samples 1.csv')